# Shark attacks data cleaning

- Importación de las librerías necesarias

In [1]:
import pandas as pd
import numpy as np
import re
import Src.SharkFn as Fn

- Creación del data frame Madre

In [2]:
df_Madre = pd.read_csv('Input/GSAF5.csv', encoding = "ISO-8859-1")

In [3]:
df_Madre.columns.values

array(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area',
       'Location', 'Activity', 'Name', 'Sex ', 'Age', 'Injury',
       'Fatal (Y/N)', 'Time', 'Species ', 'Investigator or Source', 'pdf',
       'href formula', 'href', 'Case Number.1', 'Case Number.2',
       'original order', 'Unnamed: 22', 'Unnamed: 23'], dtype=object)

In [4]:
print (df_Madre.shape)

(5992, 24)


In [5]:
null_cols = df_Madre.isnull().sum()
null_cols
null_cols[null_cols > 0]

Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
href formula                 1
href                         3
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

In [6]:
set(df_Madre['Activity']) # **!**
# set(df_Madre['Time']) # **!**
# set(df_Madre['Species ']) # **!**
# set(df_Madre['Country']) # **!**

# print(df_Madre['Type'].value_counts())
# print(df_Madre['Sex '].value_counts())
# print(df_Madre['Fatal (Y/N)'].value_counts())

# set(df_Madre['Date'])
# set(df_Madre['Year'])

# set(df_Madre['Area'])
# set(df_Madre['Location'])

# set(df_Madre['Name'])
# set(df_Madre['Age'])
# set(df_Madre['Injury'])

    #-----------#
#          set(df_Madre['Case Number'])
        
        # set(df_Madre['Investigator or Source'])
        # set(df_Madre['pdf'])
        # set(df_Madre['href formula'])
        # set(df_Madre['href'])
        # set(df_Madre['Case Number.1'])
        # set(df_Madre['Case Number.2'])
        # set(df_Madre['original order'])
        # set(df_Madre['Unnamed: 22'])
        # set(df_Madre['Unnamed: 23'])

{nan,
 'Spearfishing or fishing',
 'Skindiving for specimens',
 'On fishing boat & trailing hand in the water',
 'Bathing close inshore',
 'Floating',
 'Diving, recovering fish killed by dynamite',
 'Floating on back',
 'Surfing, fell off surfboard',
 'ship M.V. Rizal sank during typhoon',
 'Spearfishing, holding mesh bag with speared fish',
 'Swimming from capsized pirogue',
 'Fishing for tunny',
 'Competing in a light tackle game fishing',
 "Bathing in 5' of water",
 'Jumped overboard from Norwegian steamship Venator',
 "Ship's boat capsized in squall, captain & 2 sailors clinging onto hull",
 'Washing',
 'Fishing for yellowtail',
 'Dived onto shark from floating stage',
 'Washing clothes on stairs',
 'Attempted to return injured shark to the sea',
 'British ship Macedon was thrown on her beam ends by a sudden squall',
 'Free diving, hunting crayfish',
 'Lifeboat capsized',
 'Kneeling in the water',
 'Scuba Diving',
 'Swimming ashore from fishing boat swamped and sunk by a squall',
 

In [7]:
to_drop = ['Investigator or Source','pdf','href formula','href',
           'Case Number.1','Case Number.2','original order','Unnamed: 22','Unnamed: 23']
df_raw = df_Madre
df_raw.drop(to_drop, inplace=True, axis=1)


In [8]:
df_raw.rename(columns = {'Species ': 'Species',
                        'Sex ': 'Sex',
                        'Fatal (Y/N)' : 'Fatal'}, inplace=True)
df_raw.columns.values

array(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area',
       'Location', 'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal',
       'Time', 'Species'], dtype=object)

Limpiando Sex

In [9]:
df_raw.Sex = Fn.clean_last_ws(df_raw.Sex.str)

df_raw.loc[(df_raw.Sex == '.')|
           (df_raw.Sex == 'N')|
           (df_raw.Sex == 'lli'), 'Sex'] = np.nan

df_raw.Sex.value_counts(dropna=False)


M      4837
F       585
NaN     570
Name: Sex, dtype: int64

Limpiando Fatal

In [10]:
df_raw.Fatal = Fn.clean_first_ws(df_raw.Fatal.str)
df_raw.Fatal = Fn.clean_last_ws(df_raw.Fatal.str)
df_raw.Fatal = df_raw.Fatal.str.capitalize()
df_raw.loc[(df_raw.Fatal == '#value!')|
           (df_raw.Fatal == 'F')|
           (df_raw.Fatal =='Unknown'), 'Fatal'] = np.nan

df_raw.Fatal.value_counts(dropna=False)

N      4325
Y      1552
NaN     115
Name: Fatal, dtype: int64

Limpiando Species

In [11]:
df_raw.Species = Fn.clean_weird_ch(df_raw.Species.str)
df_raw.Species = Fn.clean_1to2_ch(df_raw.Species.str)
df_raw.Species = Fn.clean_num(df_raw.Species.str)
df_raw.Species = Fn.clean_mult_ws(df_raw.Species.str)
df_raw.Species = Fn.clean_last_ws(df_raw.Species.str)
df_raw.Species = Fn.clean_first_ws(df_raw.Species.str)

# set(df_raw['Species'])

In [12]:
df_raw.loc[(df_raw.Species == ''), 'Species'] = np.nan

df_raw.Species.fillna('shark', inplace = True)
df_raw.Species = df_raw.Species.str.lower()

df_raw.Species = df_raw.Species.str.findall(r"\w+\sshark")

lst_cc_sp = []
for e in df_raw.Species:
    if e == []:
        lst_cc_sp.append("undef. shark")
    else:
        lst_cc_sp.append(e[0])

df_raw["Shark_sp"] = lst_cc_sp
# df_raw.Shark_sp.value_counts()

df_raw.loc[(df_raw.Shark_sp == 'to shark')|
           (df_raw.Shark_sp == 'small shark')|
           (df_raw.Shark_sp == 'm shark')|
           (df_raw.Shark_sp == 'involve shark')|
           (df_raw.Shark_sp == 'kg shark')|
           (df_raw.Shark_sp =='lb shark'), 'Shark_sp'] = "undef. shark"

df_raw.Shark_sp.value_counts().head(10)

undef. shark        4043
white shark          614
tiger shark          259
bull shark           165
nurse shark           94
reef shark            67
whaler shark          65
blacktip shark        64
mako shark            53
hammerhead shark      44
Name: Shark_sp, dtype: int64

Limpiando Activity

In [13]:
df_raw.Activity.fillna('undefing', inplace = True)
df_raw.Activity = df_raw.Activity.str.lower()

df_raw.Activity = df_raw.Activity.str.findall(r"\w+ing")


df_raw.Activity

lst_cc_act = []
for e in df_raw.Activity:
    if e == []:
        lst_cc_act.append("undef. act")
    else:
        lst_cc_act.append(e[0])

df_raw["Deadly_act"] = lst_cc_act


df_raw.loc[(df_raw.Deadly_act == 'spearfishing')|
           (df_raw.Deadly_act == 'collecting')|
           (df_raw.Deadly_act == 'netting'), 'Deadly_act'] = "fishing"

df_raw.loc[(df_raw.Deadly_act == 'bathing')|
           (df_raw.Deadly_act == 'lifesaving')|
           (df_raw.Deadly_act == 'attempting')|
           (df_raw.Deadly_act == 'splashing')|
           (df_raw.Deadly_act == 'wading'), 'Deadly_act'] = "swimming"

df_raw.loc[(df_raw.Deadly_act == 'snorkeling')|
           (df_raw.Deadly_act == 'skindiving')|
           (df_raw.Deadly_act == 'filming')|
           (df_raw.Deadly_act == 'freediving'), 'Deadly_act'] = "diving"

df_raw.loc[(df_raw.Deadly_act == 'standing')|
           (df_raw.Deadly_act == 'floating')|
           (df_raw.Deadly_act == 'treading')|
           (df_raw.Deadly_act == 'walking')|
           (df_raw.Deadly_act == 'sinking')|
           (df_raw.Deadly_act == 'boating')|
           (df_raw.Deadly_act == 'boarding'), 'Deadly_act'] = "sailing"

df_raw.loc[(df_raw.Deadly_act == 'kayaking')|
           (df_raw.Deadly_act == 'skiing')|
           (df_raw.Deadly_act == 'playing')|
           (df_raw.Deadly_act == 'canoeing')|
           (df_raw.Deadly_act == 'windsurfing')|
           (df_raw.Deadly_act == 'rowing')|
           (df_raw.Deadly_act == 'paddling')|
           (df_raw.Deadly_act == 'sitting'), 'Deadly_act'] = "surfing"

df_raw.loc[(df_raw.Deadly_act == 'undefing')|
           ((df_raw.Deadly_act != 'fishing')&
            (df_raw.Deadly_act != 'diving')&
            (df_raw.Deadly_act != 'sailing')&
            (df_raw.Deadly_act != 'surfing')&
           (df_raw.Deadly_act != 'swimming')), 'Deadly_act'] = "undef. act"


df_raw.Deadly_act.value_counts().head(6)


swimming      1430
undef. act    1238
surfing       1198
fishing       1105
diving         622
sailing        399
Name: Deadly_act, dtype: int64

In [14]:
lst_shark = ["white shark", "tiger shark", "bull shark", "nurse shark", "reef shark", "undef. shark"]
lst_act = ["swimming", "surfing", "fishing", "diving", "sailing", "undef. act"]


df_new = df_raw.loc[ df_raw.Shark_sp.isin(lst_shark), ['Shark_sp', 'Deadly_act'] ]

df_new.groupby('Shark_sp').Deadly_act.value_counts()



Shark_sp      Deadly_act
bull shark    swimming        48
              fishing         41
              surfing         32
              undef. act      18
              diving          13
              sailing         13
nurse shark   fishing         30
              diving          23
              undef. act      22
              swimming        11
              surfing          6
              sailing          2
reef shark    fishing         28
              diving          19
              surfing          7
              swimming         6
              undef. act       6
              sailing          1
tiger shark   surfing         68
              fishing         46
              swimming        44
              diving          40
              undef. act      35
              sailing         26
undef. shark  swimming      1099
              undef. act     986
              surfing        735
              fishing        623
              diving         320
              sail

In [15]:
1&1

1

In [16]:
# df_raw.Sex = df_raw.Sex.str.replace("\s$", "")
# print(df_Madre['Sex '].value_counts())

In [17]:
text = 'bla bla diving sharking blabla'

textCC = re.findall(r'\w+ing', text)
textCC

['diving', 'sharking']

In [18]:
# df_raw = df_Madre

# def fun(*args,**kwargs):
#     print("Generic FN:")
#     print("\tArgs", args)
#     print("\tKwargs", kwargs)
    
# def clean_last_ws (serie):
#     aux = re.sub("\s$", "", serie)
#     return aux
    
# new = df_raw[["Sex "]].values.tolist()
# new
# aux = re.sub("\s$", "", *new)
# aux